# Recursive Inverted Index (5 Points)

Copyright [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

In this exercise, you will implement an [inverted index](https://en.wikipedia.org/wiki/Inverted_index). An inverted index is a mapping from the content (e.g. words) of
a document to the document itself. In addition, in this exercise we radix-partition the words, i.e. we _recursively_ partition the words characterwise similar to the notebook [Indexing by Recursive External Partitioning.ipynb](https://github.com/BigDataAnalyticsGroup/bigdataengineering/blob/master/Indexing%20by%20Recursive%20External%20Partitioning.ipynb). This structure allows us to easily search for documents that contain a word that exactly matches a given search word, but also words that start with the given search word (prefix search). Implement the following functions of the class `RecursiveInvertedIndex` in the attached notebook:
* `__init__`: This is the constructor of the class, which takes a list of documents names and creates the
    recursive, inverted index based on these documents. Your words must be radix-partitioned. You can use the helper
        function`get_words_from_document(document_name)`, which takes a document name as input and then
        returns a set of words contained in the document. Note, that all words are converted to lowercase.
* `search(word)`: This function uses the inverted index to return a list of all matching documents given the key `word`. Matching here means that the search word either exactly matches one of the words in a document, or at least one of the words in a document starts with the search word. If no documents exist for a search word, an empty set is returned. Otherwise, the set of all matching documents is returned. Your implementation should be case-insensitive.

Your implementation must pass all provided unit tests without hardcoding!

In [2]:
def get_words_from_document(document_name):
    with open(document_name) as f:
        text = f.read().replace('.', '')
        word_list = text.split(" ")
        words = set([word.lower() for word in word_list])
        return words

In [3]:
# A class implementing a recursive, inverted index
class RecursiveInvertedIndex:

    # Dictionary to store the word-to-document mapping
    word_dict = {}

    def create_key_values(self, file_list):
        """
        Create a dictionary where each key is a word and the value is a set of filenames 
        that contain the word.
            
        Returns:
            dict: Dictionary with words as keys and sets of filenames as values.
        """
        output_dict = {}

        # Process each file in the file list
        for file in file_list:
            words = get_words_from_document(file)  # Extract words from the document
            for word in words:
                if word not in output_dict:
                    output_dict[word] = set()  # Initialize set for new word
                output_dict[word].add(file)  # Add the file to the set for the word

        return output_dict

        
    def radix_indexer(self):
        """
        Create a recursive index by generating substrings of words and mapping them 
        to the sets of documents containing the original words.
        """
        temp_word_dict = {}
        word_dict = self.word_dict

        # Iterate through each word and its associated document set
        for word, doc_set in word_dict.items():
            # Generate all substrings of the word
            for i in range(1, len(word) + 1):
                substring = word[:i]  # Create a substring from the beginning to the ith character
                if substring not in temp_word_dict:
                    temp_word_dict[substring] = set()  # Initialize set for new substring
                temp_word_dict[substring].update(doc_set)  # Add the document set to the substring

        self.word_dict = temp_word_dict  # Update the word_dict with the new recursive index

    def __init__(self, document_names):
        """
        Initialize the RecursiveInvertedIndex with a list of document names. (Constructor)
        """
        self.word_dict = self.create_key_values(document_names)  # Create the initial word-to-document mapping
        self.radix_indexer()  # Perform the radix indexing to generate substrings

    def search(self, word): 
        """
        Search for documents containing the exact word or any word that starts with the given word.
        
        Returns:
            set: Set of filenames containing the word or any word starting with the given word.
        """
        documents = set()
        for key, value in self.word_dict.items():
            if word.lower() == key.lower():
                documents.update(value)  # Add documents containing the word to the result set
        return documents

# Hier nur Ausgabe als Test

In [4]:
test = RecursiveInvertedIndex(["data/indexes/test_file_1.txt", "data/indexes/test_file_2.txt", "data/indexes/test_file_3.txt"])
c = test.word_dict
for key, value in c.items():
    print(key + " -> " + ", ".join(map(str, value)))

i -> data/indexes/test_file_1.txt, data/indexes/test_file_3.txt, data/indexes/test_file_2.txt
a -> data/indexes/test_file_1.txt, data/indexes/test_file_3.txt
b -> data/indexes/test_file_1.txt, data/indexes/test_file_3.txt
bd -> data/indexes/test_file_1.txt, data/indexes/test_file_3.txt
bde -> data/indexes/test_file_1.txt, data/indexes/test_file_3.txt
c -> data/indexes/test_file_1.txt, data/indexes/test_file_2.txt
co -> data/indexes/test_file_1.txt, data/indexes/test_file_2.txt
cou -> data/indexes/test_file_1.txt, data/indexes/test_file_2.txt
cour -> data/indexes/test_file_1.txt, data/indexes/test_file_2.txt
cours -> data/indexes/test_file_1.txt, data/indexes/test_file_2.txt
course -> data/indexes/test_file_1.txt, data/indexes/test_file_2.txt
l -> data/indexes/test_file_1.txt, data/indexes/test_file_2.txt
lo -> data/indexes/test_file_1.txt
lot -> data/indexes/test_file_1.txt
in -> data/indexes/test_file_1.txt, data/indexes/test_file_2.txt
t -> data/indexes/test_file_1.txt, data/indexes/

# Hier einfach ein paar Test ausgeführt

In [8]:
print("BDE")
print(test.search("BDE"))
print("DBSys")
print(test.search("DBSys"))
print("i")
print(test.search("i"))
print("I")
print(test.search("I"))

BDE
{'data/indexes/test_file_1.txt', 'data/indexes/test_file_3.txt'}
DBSys
{'data/indexes/test_file_3.txt', 'data/indexes/test_file_2.txt'}
i
{'data/indexes/test_file_1.txt', 'data/indexes/test_file_3.txt', 'data/indexes/test_file_2.txt'}
I
{'data/indexes/test_file_1.txt', 'data/indexes/test_file_3.txt', 'data/indexes/test_file_2.txt'}


### Unit tests

Note that test cases are by no means exhaustive!

In [6]:
import unittest

class RecursiveInvertedIndexTest(unittest.TestCase):
    
    def setUp(self):
        self.documents = ["data/indexes/test_file_1.txt", "data/indexes/test_file_2.txt", "data/indexes/test_file_3.txt"]
        self.rec_inverted_index = RecursiveInvertedIndex(self.documents)
    
    def test_no_result(self):
        search_result = self.rec_inverted_index.search("Im")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search("BDER")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search("lotin")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search("")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search(" ")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search("ar")
        self.assertEqual(len(search_result), 0)
    
    def test_with_result(self):
        search_result = self.rec_inverted_index.search("bde")
        self.assertEqual(len(search_result), 2)
        self.assertTrue("data/indexes/test_file_1.txt" in search_result and "data/indexes/test_file_3.txt" in search_result)
        search_result = self.rec_inverted_index.search("DBS")
        self.assertEqual(len(search_result), 2)
        self.assertTrue("data/indexes/test_file_2.txt" in search_result and "data/indexes/test_file_3.txt" in search_result)
        search_result = self.rec_inverted_index.search("I")
        self.assertEqual(len(search_result), 3)
        for document in self.documents:
            self.assertTrue(document in search_result)
        search_result = self.rec_inverted_index.search("course")
        self.assertEqual(len(search_result), 2)
        self.assertTrue("data/indexes/test_file_1.txt" in search_result and "data/indexes/test_file_2.txt" in search_result)
        search_result = self.rec_inverted_index.search("o")
        self.assertEqual(len(search_result), 1)
        self.assertTrue("data/indexes/test_file_3.txt" in search_result)

In [7]:
# Run the unit test without shutting down the jupyter kernel
unittest.main(argv=['ignored', '-v'], verbosity=2, exit=False)

test_no_result (__main__.RecursiveInvertedIndexTest) ... ok
test_with_result (__main__.RecursiveInvertedIndexTest) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK
